## 2. **Deep Learning for Computer Vision**
### 2.1 Introduction
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/lecture_1_feifei.pdf_page_06.png?raw=true)

    이제 컴퓨터비젼에서 활용되는 딥러닝 기술들을 살펴보겠습니다.
    컴퓨터 비젼에 활용되는 현재 딥러닝 핵심 네트워크는 Convolutional Nerual Network입니다.
    Covolutional Neural Network는 CNN, ConvNet이라고 대부분 줄여서 부르고 있습니다.
    CNN의 등장은 컴퓨터비전의 다양한 부분에 영향을 줬습니다.
    객체검출.......VQA(시각적 질의응답)까지 전체적인 정확도 향상을 가져왔습니다.
***

## Preparation

     colab -> 런타임 -> 런타임 유형 변경 -> GPU -> 저장

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import pandas as pd
import seaborn as sns
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy # Add Deepcopy for args
import matplotlib.pyplot as plt
from IPython.display import HTML

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
partition = {'train': trainset, 'val':valset, 'test':testset}

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

def plot_loss_variation(result, **kwargs):

    '''
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    '''
    list_data = []
    for epoch, train_loss in enumerate(result['train_losses']):
        list_data.append({'type': 'train', 'loss': train_loss, 'epoch': epoch})
    for epoch, val_loss in enumerate(result['val_losses']):
        list_data.append({'type': 'val', 'loss': val_loss, 'epoch': epoch})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'loss', marker='.')
    g.add_legend()
    g.fig.suptitle('Train loss vs Val loss')
    plt.subplots_adjust(top=0.89)


def plot_acc_variation(result, **kwargs):
    list_data = []
    for epoch, train_acc in enumerate(result['train_accs']):
        list_data.append({'type': 'train', 'Acc': train_acc, 'test_acc': result['test_acc'], 'epoch': epoch})
    for epoch, val_acc in enumerate(result['val_accs']):
        list_data.append({'type': 'val', 'Acc': val_acc, 'test_acc': result['test_acc'], 'epoch': epoch})


    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'Acc', marker='.')

    def show_acc(x, y, metric, **kwargs):
        plt.scatter(x, y, alpha=0.3, s=1)
        metric = "Test Acc: {:1.3f}".format(list(metric.values)[0])
        plt.text(0.05, 0.95, metric, horizontalalignment='left', verticalalignment='center',
                 transform=plt.gca().transAxes, bbox=dict(facecolor='yellow', alpha=0.5, boxstyle="round,pad=0.1"))

    g = g.map(show_acc, 'epoch', 'Acc', 'test_acc')

    g.add_legend()
    g.fig.suptitle('Train Accuracy vs Val Accuracy')
    plt.subplots_adjust(top=0.89)

def train(net, partition, optimizer, criterion, args):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                              batch_size=args.train_batch_size,
                                              shuffle=True, num_workers=0)
    net.train()
    optimizer.zero_grad()

    correct = 0
    total = 0
    train_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc


def validate(net, partition, criterion, args):
    valloader = torch.utils.data.DataLoader(partition['val'],
                                            batch_size=args.test_batch_size,
                                            shuffle=False, num_workers=0)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc


def test(net, partition, args):
    testloader = torch.utils.data.DataLoader(partition['test'],
                                             batch_size=args.test_batch_size,
                                             shuffle=False, num_workers=0)
    net.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc


def experiment(partition, args):
    if args.model == 'CNN':
        net = CNN()
    elif args.model == 'Resnet':
        net = Resnet()
    else:
        raise ValueError('In-valid model choice')
    net.cuda()

    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')

    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args)
        val_loss, val_acc = validate(net, partition, criterion, args)
        te = time.time()

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        print(
            'Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch,
                                                                                                                  train_acc,
                                                                                                                  val_acc,
                                                                                                                  train_loss,
                                                                                                                  val_loss,
                                                                                                                  te - ts))

    test_acc = test(net, partition, args)

    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    return vars(args), result

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

print('ready.')

### 2.2 What is Convolutional Neural Network?
#### 2.2.1 How human recognize an image?
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide27.PNG?raw=true)

    Neural Net 강의에서 뉴런이 항상 등장하듯이, CNN강의에서는 대부분 이 그림을 보실 수 있는데요,
    과연 인간의 뇌가 MLP처럼 단순하게 동작할까? 좀 더 우리가 모르는 것이 있지 않을까?
    사람이 이미지를 어떻게 인식하는지 생물학적(biologic)으로 알아보기 시작합니다.
    그래서  저런 단순한 움직임을 보여주며, 고양이의 뇌를 관찰합니다.
    선이 움직인다든지, 원이 커진다든지 이런 단순한 움직임을 보여줬는데.
    이런 움직임에서는 이 뉴런들만 활성화되고, 이런 움직임에서는 이런 뉴런들만 활성화되는것을
    발견할 수 있었습니다.
    그래서 우리는 이미지를 볼때 이런 간단한 특징들을 담당하는 뉴런들이 있는게 아닐까라는 추측을 하게됩니다.
***

#### 2.2.1 How human recognize an image?
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide28.PNG?raw=true)

    그리고 더 연구를 해보니까, 이 뉴런들이 계층 구조를 이루고 있다는 것을 알게됬거든요?
    무작위하게 얽혀있는게 아니라, 시각세포와 연결되어 있는 이런 얕은(shallow)한 뉴런들이 있고
    그리고 뒤에 이런 딥한 뉴런들이 있었는데, 이런 뉴런들이 언제 활성화가 되는지 살펴보니까
    이런 얕은 얘들은 빛이 있냐 없냐, 어떻게 기울어져있냐 등 간단한 정보에 반응을하고
    깊어지면 깊어질수록 이 선이 움직이느냐, 선이 이어져있느냐 끝나느냐.
    깊어지면 깊어질수록 고차원적인 정보를 담당하는 것을 알아내게 됩니다.
    그럼 이것을 어떻게 수학적으로 모델링해서 NN으로 만들 수 있을까? 이걸 고민해볼 차례겠죠.
***

In [ ]:

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FwFduRA_L6Q" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

    얀 르쿤 교수님 팀의의 첫 번째 lenet test 영상인데요,
    얀 르쿤 교수님 팀에서 lenet이라는 convolutional network를 발표합니다.
    영상에서 처럼 hand-writting 문자를 성공적으로 인식하는 것을 보실 수 있습니다.
    convolution, pooling등을 이용하며 이미지를 어떻게 딥러닝에 학습시킬지 방법론을 최초로 정립하게 됩니다.
***

### 2.3 Convolutional Neural Network
#### 2.3.1 How can we feed images to a neural network?
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide31.PNG?raw=true)

    그렇다면 이미지를 어떻게 뉴럴넷에 넣을 수 있을까요?
    이런 이미지가 있다고 생각해봅시다.
    배경에 나무라는 객체가 있는 심플한 이미지 입니다.
***

#### 2.3.1 How can we feed images to a neural network?
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide32.PNG?raw=true)

    이 이미지를 뉴럴넷에 입력으로 넣고 싶은데, 뉴럴넷은 입력으로 벡터를 받죠.
    그런데 이미지는 x,y의 two-디멘션 그리드의 픽셀들입니다.
***

#### 2.3.1 How can we feed images to a neural network?
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide34.PNG?raw=true)

    그래서 뉴럴넷에 넣기 위해 픽셀들을 vectorize해서 벡터로 만듭니다.
    이제 이 쫙 핀 이미지를 뉴럴넷에 넣으면 되겠네요.
***

#### 2.3.1 How can we feed images to a neural network?
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide35.PNG?raw=true)

    하지만 기대와는 다르게 이렇게 넣으면 안됩니다.
    첫번째 문제는 이미지안에서 나무가 조금만 이동하더라도 인풋에 대한 특징이 달라지게 되어
    전혀다른 결과물을 만들게 됩니다.
***

#### 2.3.1 How can we feed images to a neural network?
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide36.PNG?raw=true)

    우리는 이미지안에서 객체가 어느위치에 있든 같은 객체라고 인식하길 원합니다.
    이것을 Translation invariance하다고 하며,
    의미있는 정보를 위치에 상관없이 추출하고 같은 결과를 얻길 원합니다.
***

#### 2.3.2 Convolution ≈  Cross-correlation
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide37.PNG?raw=true)

    객체의 위치에 구애받지 않고, 지역적 특징을 잘 추출하기 위해서
    여기서 Convolution 연산을 사용합니다.
    이해를 돕기 위해서 이미지를 1차원으로 줄였다고 가정하고 설명하겠습니다.
    Convolution 연산의 수학적 의미는 왼쪽과 같고,
    쉽게 한칸씩 이동하면서 커널(필터)를 뒤집고 곱해서 더한다라고 생각하시면됩니다.
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide38.PNG?raw=true)
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide39.PNG?raw=true)
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide40.PNG?raw=true)
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide41.PNG?raw=true)

    인풋 시그널이 어디에 있든 result의 위치는 다르겠지만 같은 형태의 결과가 나올 것입니다.
    이렇게 convolution연산과 softmax를 통해서 Translation invariance한 결과를 만듭니다.

    실제 뉴럴넷에서는 커널을 뒤집는것이 중요하지 않기 때문에, 커널을 그냥 곱해주는
    cross-correlation연산을 하지만 convolution의 개념적인 것을 사용한 것이기 때문에
    cross-correlation neural net이라부르지 않고 Convolutional neural net이라고 부릅니다.

    딥러닝에서는 아래 result를 가지고 우리가 원하는 정답값과 비교를 해가면서 차이를 줄이는 방향으로
    위에 kernel형태를 학습을 반복하면서 변경해 나갑니다.
***

#### 2.3.3 Convolution operation
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/sobel.png?raw=true)

    이건 sobel 필터라는 건데요 sobel이란 사람이 윤곽선 검출을 위해 kernel(필터)를 이렇게 정의를 한겁니다.
    위쪽필터를 전체이미지에 돌리면 왼쪽과 오른쪽의 차이를 저장하기 때문에 세로의 성분을 추출할 수 있을 것이구요
    아래쪽 필터는 반대로 가로 성분을 추출할 수 있습니다.
    이 두개를 합치면 flat한 부분을 제외한 윤곽선을 추출할 수 있겠죠?
***

In [ ]:
# haar like
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/hPCTwxF0qf4" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')


    마찬가지로 이건 Viola, Jones가 정의한 haar-like feature인데요,
    얼굴에 대한 특징을 여러개의 필터 이용해서 찾는 것을 보실 수 있습니다.
    딥러닝에서는 어떻게 하면 목표 객체의 특징을 잘 찾을 수 있는 커널을
    사람이 여러가지 실험을 통해서 발견하는 것이 아니라.
    컴퓨터가 찾아주는 것이죠.

#### 2.3.3 Convolution operation
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide43.PNG?raw=true)

    그럼 다시 2차원 이미지로 돌아와봅시다.
    1차원때 설명한 것과 같은 방법으로 convlution 진행합니다.
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide44.PNG?raw=true)
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide45.PNG?raw=true)

    한칸씩 이동하면서 곱하고 더해서 값을 만들고, 오른쪽 끝에 도달하면 한칸 아래로 내려와서
    반복합니다. 이것을 이미지 끝까지 진행하면 오른쪽과 같은 2차원 결과가 만들어질 것입니다.
    참고로 말씀드리자면 한칸씩 이동하면 stride를 1이라고 하고, 2칸씩 이동하면 stride를 2라고 합니다.
    즉 stride는 kernel을 몇 칸씩 이동했느냐는 의미입니다.
***

#### 2.3.3 Convolution operation
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide46.PNG?raw=true)

    우리는 3x3의 슬라이딩 윈도우를 커널이라고 부르고,
    이 커널과 곱하고 합해져서 나온 값을 feature map이라고 부릅니다.
    그리고 이 feature map을 보시면
    왼쪽 위 한칸이 상위의 3x3의 정보가 들어온것이기 때문에,
    이 칸의 Receptive filed, 즉 이 칸이 수용하고 있는 정보는 여기 3x3이다 라고 말합니다.

***

#### 2.3.3 Convolution operation
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide47.PNG?raw=true)

    이 convolution unit들은 뉴럴넷의 뉴런과 같은 기능을 하고 있으며,
    weight와 bias값을 가지고 이미지에서 선, 색상등 다양한 feature를 잘 추출하기 위해 학습되어 집니다.

    그럼 이미지를 하나 불러서 convolution 연산을 실습해봅시다.
***

In [ ]:
# select car image in cifar10 dataset
img = partition['train'].dataset.data[4]
plt.imshow(img)
plt.show()
print('1.image shape:',img.shape)

# trans np to torch (h,w,c) -> (c,h,w)
x = transform(img)
print('2.tensor shape:',x.shape)

conv = nn.Conv2d(3, 3, 3, stride=1)
# add batch dim
x = x.unsqueeze(0)
x = conv(x)
imshow(x.squeeze().detach().cpu())
print('3.shape of feature map after conv:',x.squeeze().shape)

#### 2.3.3 Convolution operation
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide52.PNG?raw=true)

    방금 실습에서는 3개의 커널을 사용해서 convolution연산을 했습니다.
    커널이 많아질수록 각 커널이 예를들어 가로, 세로, 대각선 특징들을 뽑듯이
    다양한 feature들을 추출할 수 있습니다.
***

#### 2.3.3 Convolution operation
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide55.PNG?raw=true)

    그런데 위 코드에서 보신 것처럼 convolution연산은 convolution을 거듭할 수록 다음
    feature맵이 작아지게 됩니다.
    입력 이미지가 작을 경우에는 convolution 연산을 몇번 할 수 없겠죠?
    마찬가지로 입력이 계속 작아지니 네트워크도 깊게 쌓을 수 없을 것 입니다.
***

#### 2.3.4 Padding
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide57.PNG?raw=true)

    그래서 convolution 연산을 할 때, 테두리에 여분을 추가해서
    feature map이 축소되는 것을 방지합니다.
    이렇게 여분을 추가하는 것을 padding이라 부르고,
    input과 output크기가 같게 나오도록 conv하는 것을 same convolution이라고 부릅니다.
    3x3 커널일때는 한줄만 추가하면 되지만, 5x5 커널을 사용한다면 두줄을 추가해야 same conv가 되겠죠?
***

In [ ]:
# select car image in cifar10 dataset
x = transform(img)
print(x.shape)

conv = nn.Conv2d(3, 3, 3, stride=1, padding=1)
x = x.unsqueeze(0)
x = conv(x)
print(x.squeeze().shape)
x = conv(x)
print(x.squeeze().shape)
x = conv(x)
print(x.squeeze().shape)

#### 2.3.5 Pooling
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide59.PNG?raw=true)

    반대로 입력 이미지가 너무 클 경우를 생각해봅시다.
    입력이 크다는 것은 유지해야할 parameter가 많아진다는 것을 뜻하는 것이고,
    이는 모델이 너무 커져 학습속도도 느려지고 같은 파라미터값들이 많아지는 현상이 일어납니다.
    그래서 우리는 어느정도 이 feature map크기에서 특징추출을 한것 같다라는 생각이들면
    pooling을 통해서 feature map을 줄여줍니다. 보통 계산하기 편하기 위해 입력을 반으로 줄입니다.
***

In [ ]:
# select car image in cifar10 dataset
x = transform(img)
print('1.input shape:',x.shape)

conv = nn.Conv2d(3, 3, 3, stride=1, padding=1)
pooling = nn.MaxPool2d(2,stride=2)
x = x.unsqueeze(0)
x = pooling(x)
print('2.after pooling:', x.squeeze().shape)

#### 2.3.6 Sequence of convolutional layers
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide61.PNG?raw=true)
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide63.PNG?raw=true)
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide64.PNG?raw=true)

    위와같이 CNN은 Convolution, Activation Fuction인 ReLU, Pooling등을 반복하면서 연산을 해나갑니다.

    첫번째 Convolution Kernel들은 색상이나 선의 방향등 단순한 feature들을 학습하고,
    Layer가 깊어질수록 더이상은 인간이 이해하지 못하는 고차원적인 feature들이 representation됩니다.RGB기 때문에..
    그래도 깊이 있는 애들이 복잡한 것을 탐지한다고 예상할 순 있겠네요.

    shallow한곳에서는 단순한 feature를 인식하고 deep한곳에서는 간단한 정보를 취합함으로써
    좀 더 복잡한 정보를 탐지한다.
    이런식으로 뇌가 동작한다고 연구가 되어왔는데 그것을 잘 모방했다고 볼 수 있겠네요.

    이 그림은 언제 Layer5의 뉴런들이 Activation되는지 weight를 쌓아가면서 계산하여 시각화을 한건데요,
    Layer 5까지오면 꽃잎의 형태, 사람얼굴, 강아지의 얼굴등 고차원적인 정보를 다루는 것을 볼 수 있습니다.

    그럼 conv, relu, pooling을 조합한 연속적인 conv연산의 실습을 해볼까요?
***

In [ ]:
# select car image in cifar10 dataset
img = partition['train'].dataset.data[4]
plt.imshow(img)
plt.show()

# trans np to torch (h,w,c) -> (c,h,w)

x = transform(img)

conv = nn.Conv2d(3, 3, 3, stride=1, padding=1)
relu = nn.ReLU()
pooling = nn.MaxPool2d(2,2)
# add batch dim
x = x.unsqueeze(0)
x = relu(conv(x))
imshow(x.squeeze().detach().cpu())
x = pooling(relu(conv(x)))
imshow(x.squeeze().detach().cpu())
x = relu(conv(x))
imshow(x.squeeze().detach().cpu())

    여기서는 커널(필터)들이 어떻게 학습되어지는진 볼 순 없지만
    Conv연산을 거칠수록 feature맵이 어떻게 변화되는지 살펴 볼 수 있습니다.

#### 2.3.6 Sequence of convolutional layers
![pt](https://github.com/tiktakdad/dl4cv/blob/master/images/pt/Slide65.PNG?raw=true)

    이것은 Convolution Net의 기본 구조를 나타낸건데요. 한번 봅시다.
    아까 말씀드렸듯이, Conv, ReLU, Pool을 반복하고, 마지막에 FC가 있네요.
    FC는 MLP와 같은 말입니다. 마지막에 나온 고차원적인 정보를 vectorize 시켜서
    MLP에 넣어주고, 이게 어떤 객체인지 MLP와 했던것과 똑같이 분류문제를 풉니다.
    마지막 feature맵은 image의 고차원적인 feature이기 때문에 MLP로 넣을 수 있게 되는거구요.

    그럼 위와 그림과 같은 ConvNet을 하나 구현해볼까요?
***

In [ ]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1 )
        self.conv1_2 = nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1)
        self.conv2_1 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1 )
        self.conv2_2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv3_1 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        #32->16->8->4 : 4*4*64
        self.fc = nn.Linear(4*4*64, 10)

    def forward(self, x):
        x = self.relu(self.conv1_1(x))
        x = self.relu(self.conv1_2(x))
        x = self.pool(x)
        x = self.relu(self.conv2_1(x))
        x = self.relu(self.conv2_2(x))
        x = self.pool(x)
        x = self.relu(self.conv3_1(x))
        x = self.relu(self.conv3_2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "cifar-10"
#models = ['CNN', 'Resnet']
args.model = 'CNN'
args.act = 'relu'
args.l2 = 0.00001
args.optim = 'SGD'  # 'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 1e-3
args.epoch = 40

args.train_batch_size = 128
args.test_batch_size = 32


print(args)
setting, result = experiment(partition, deepcopy(args))
plot_loss_variation(result)
plot_acc_variation(result)
